In [1]:
from datasets import load_dataset

DATASET_NAME = "qasc"
dataset = load_dataset("allenai/qasc", "", split=["train", "validation",'test'])

In [6]:
# store_corpus=[]
# with open('QASC_Corpus.txt', 'r') as fr:
#     while True:
#         content=fr.readline()
#         store_corpus.append(content)
#         if not content:
#             break
# print(len(store_corpus))

16987131


In [2]:
dataset[2][0]

{'id': '3C44YUNSI1OBFBB8D36GODNOZN9DPA',
 'question': 'What type of birth do therian mammals have?',
 'choices': {'text': ['Anthax',
   'under water',
   'uterus',
   'wombs',
   'two',
   'moles',
   'live',
   'embryo'],
  'label': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']},
 'answerKey': '',
 'fact1': '',
 'fact2': '',
 'combinedfact': '',
 'formatted_question': 'What type of birth do therian mammals have? (A) Anthax (B) under water (C) uterus (D) wombs (E) two (F) moles (G) live (H) embryo'}

In [4]:
choice_len = max([len(x["label"]) for x in dataset[1]["choices"]])
choice_len

8

In [5]:
NUM_PERMUTE_QUESTION = None
EACH_HAS = 16
NUM_CHOICE=8

In [6]:
# all_ques1=[x for x in dataset[0]['id']]
# all_ques2=[x for x in dataset[1]['id']]

In [7]:
# [x for x in all_ques1 if x in all_ques2]

In [8]:
import numpy as np


def get_prompt(dataset, index, location=-1, has_choice=False,dataset_index=0):
    is_test=dataset_index==2
    is_validation=dataset_index==1
    try:
        json_line = dataset[index]
        question = json_line["question"]
        choices = json_line["choices"]
        choice_texts = choices["text"]
        if not is_test:
            answer_key = json_line["answerKey"]
            answer_key_idx = ord(answer_key[0]) - ord("A")
            answer_text = choices["text"][answer_key_idx]
            if location>-1:
                perm=np.random.permutation(len(choice_texts)-1)
                choice_texts.remove(answer_text)
                choice_texts = [choice_texts[perm[i]].replace("\n", " ") for i in range(len(choice_texts))]
                choice_texts.insert(location, answer_text)
        if location==-1:
            perm=list(range(NUM_CHOICE))
            choice_texts = [choice_texts[perm[i]].replace("\n", " ") for i in range(len(choice_texts))]
        candidates = " ".join(
            [
                f"({label if has_choice else ' '}) {text}"
                for text, label in zip(choice_texts, choices["label"])
            ]
        ).replace("\n", " ")
        
        # print(answer_key_idx,answer_text)
        fact = ''
        if is_test:
            prompt = f"{question} \\n {candidates}"
            return [(json_line['id'],prompt)]
        else:
            fact=f"{json_line['combinedfact']}{'' if json_line['combinedfact'][-1]=='.' else '.'}"
            prompt_without_hint = f"{question} \\n {candidates}"
            prompt_with_hint = f"{fact} {prompt_without_hint}"
            if is_validation:
                # only need one version
                return [(prompt_with_hint, answer_text)]
            # if training set
            return  [(prompt_without_hint, answer_text)] #,(prompt_with_hint, answer_text)]
    except:
        raise Exception(f'Answer key is {answer_key}')
        # print("QUESTION: ", question, "ANSWER KEY", answer_key)

In [9]:
import random
from tqdm import tqdm, trange
import pickle

if NUM_PERMUTE_QUESTION:
    container = []
    each_choice_has=EACH_HAS/NUM_CHOICE
    for ques_index in trange(NUM_PERMUTE_QUESTION):
        for idx in range(EACH_HAS):
            location=int(idx/each_choice_has)
            content=get_prompt(dataset[1], ques_index, location, has_choice=False)
            # print(content, location)
            container.append(content)
            
    pickle.dump(
        (NUM_PERMUTE_QUESTION, EACH_HAS, container),
        open(f"{DATASET_NAME}_test_permute.pkl", "wb"),
    )
else:
    for index, dataname in enumerate(["train", 'validate',"test"]):
        container = []
        for ques_index, ques in enumerate(tqdm(dataset[index])):
            container.extend(get_prompt(dataset[index], ques_index, has_choice=False, dataset_index=index))
        pickle.dump(container, open(f"{DATASET_NAME}_{dataname}.pkl", "wb"))

100%|██████████| 920/920 [00:00<00:00, 3093.48it/s]
